In [ ]:
import os
import socket
import cv2
import pickle
import numpy as np
import tensorflow as tf
import argparse
import sys
from os import system,name
import subprocess as sp
from utils import label_map_util
from utils import visualization_utils as vis_util
import struct ## new


IM_WIDTH = 1280
IM_HEIGHT = 720
sys.path.append('..')


MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'

CWD_PATH = os.getcwd()

PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

PATH_TO_LABELS = os.path.join(CWD_PATH,'data','mscoco_label_map.pbtxt')

NUM_CLASSES = 15


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)


image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

num_detections = detection_graph.get_tensor_by_name('num_detections:0')


HOST=''
PORT=8089

s=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
print ('Socket created')

s.bind((HOST,PORT))
print ('Socket bind complete')
s.listen(10)
print ('Socket now listening')

conn,addr=s.accept()

### new
data = b""
payload_size = struct.calcsize("L") 
while True:
    while len(data) < payload_size:
        data += conn.recv(4096)
    packed_msg_size = data[:payload_size]
    data = data[payload_size:]
    msg_size = struct.unpack("L", packed_msg_size)[0]
    while len(data) < msg_size:
        data += conn.recv(4096)
    frame_data = data[:msg_size]
    data = data[msg_size:]
    ###
   
    frame=pickle.loads(frame_data)

   

    #For resizing the video frame use following command
    #frame = cv2.resize(frame, dsize=(100, 200), interpolation=cv2.INTER_CUBIC)

    frame_expanded = np.expand_dims(frame, axis=0)
    totalcount = 0

    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores, detection_classes, num_detections],
        feed_dict={image_tensor: frame_expanded})

    # Draw the results of the detection (aka 'visulaize the results')
    vis_util.visualize_boxes_and_labels_on_image_array(
        frame,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8,
        min_score_thresh=0.55
        )
    
    final_score = np.squeeze(scores)    
    count = 0
    for i in range(100):
        if scores is None or final_score[i] > 0.5:
                count = count + 1
    
    cv2.putText(frame,"People : "+str(count) ,(10, 35),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0, 0xFF, 0xFF),2)
    cv2.imshow('Object detector', frame)
    
    
    tmp = sp.call('clear',shell=True)
    print(count)
    # Press 'q' to quit
    if cv2.waitKey(1) == ord('q'):
        break

video.release()
cv2.destroyAllWindows()










Socket created
Socket bind complete
Socket now listening
